In [12]:
import time
import pandas as pd
import numpy as np
cluster_st1=pd.read_csv('cluster_stops1.csv') # current clustering
cluster_st1.head()
arena='Schlachthof'
depot='Hagen Wendeplatz'
stops_full=pd.read_csv('full_distance_matrix_lueneburg.csv',sep=';')
stops_sparse=pd.read_csv('sparse_matrix_lueneburg.csv',sep=';')
stops_full.shape
#cluster_st1.loc[cluster_st1['cluster']==0]

df=stops_full
df.set_index("name", inplace = True)

# list of clusters
lst=[]
for i in range(16):
    lst_i=[]
    for idx,row in cluster_st1.iterrows():
        if row['cluster']==i:
            lst_i.append(row['name'])
    lst.append(lst_i)
#remove the stop with 0 passengers    
lst.remove(lst[0])

#check if a certain stop contained in a cluster
def stInLst(lst,stop):
    for st in lst:
        if st==stop:
            return True
    return False

# define initial temperature for clusters
def tmax(lst):
    T=[]
    for st1 in lst:
        for st2 in lst:
            T.append(df.loc[st1,st2])
    return max(T)

#check if a stop appeared in a list of clusters
def checkStop(lstoflst,stop):
    i=0
    for sublst in lstoflst:
        i+=1
        if stInLst(sublst,stop):
            print('sublist nr ',i)
            return True
    return False
#checkStop(lst,arena)

#remove arena and depot from the list
#lst.remove(elmt)
for sublst in lst:
    for stop in [arena,depot]:
        if stInLst(sublst,stop)==True:
            sublst.remove(stop)
            
#two-opt neighborhood operator
# s is a list of stops
def two_opt(s,i,j):
    s_prime=[]
    if i==0:
        t1=[]
    else:
        t1=s[0:i]
    t2=s[i:(j+1)]
    t2.reverse()
    t3=s[(j+1)::]
    s_prime=t1+t2+t3
    return s_prime

#cost function: distance, lst is a list of clusters
def cost(lst):
    l = 0
    for i in range(len(lst)-1):
        l += df.loc[lst[i]][lst[i+1]]
    #l += df.loc[lst[len(lst)-1],stop] 
    return l

#get the optimal route from a list of two routes
#list_check=[(['a'],1),(['b'],2)]
def opt_sol(list_check):
    if list_check[0][1]>list_check[1][1]:
        return list_check[1]
    else:
        return list_check[0]

# add depot and arena to each route
for sublst in lst:
    sublst.insert(0,depot)
    sublst.append(arena)
    
#lst is list of clusters
def shortPath_SA_stops(lst,P,delta_P,Tmax, alpha):
    best_sol= [] #save the good iterations with improvement (does not take no_change into account if it does not improve)
    k_no_change=0
    #s = list(range(len(G)))
    c = cost(lst)
    ntrial = 1
    #T = 35 # initial 30
    T=Tmax
    alpha = 0.9
    while (ntrial <= N) and (T> T0) and (k_no_change<K):   
        for i in range(1,len(lst)-2):
            for j in range(i+1,len(lst)-1):
                s1=two_opt(lst,i,j)
                c1 = cost(s1)
                if c1 < c:
                    s, c = s1, c1
                    best_sol.append((s1,c1))
                else:
                    if np.exp(-(c1 - c)/T) > P:
                        lst, c = s1, c1
                        k_no_change=0
                    else:
                        k_no_change+=1
        T = alpha*T
        ntrial += 1
        if ntrial > N*(2/3):
            P+=delta_P
        lst_opt=[best_sol[-1], (s1,c1)]
        result=opt_sol(lst_opt)
    return result

def run_SA(lst_clusters,P,delta_P,alpha,K,T0,N):
    start = time.time()
    routes=[]
    total_route=0
    for sublst in lst:
        Tmax=tmax(sublst)
        result=shortPath_SA_stops(sublst,P,delta_P,Tmax,alpha)
        total_route+=result[-1]
        routes.append(result)
    end=time.time()
    duration=end-start
    return total_route, routes, duration

In [17]:
#single run
delta_P=0
P=0.6
N=500
T0=0.1
K=200
alpha=0.9
import time
routes=[]
total_route=0
start = time.time()
for sublst in lst:
    #duration=0
    Tmax=tmax(sublst)
    result=shortPath_SA_stops(sublst,P,delta_P,Tmax,alpha)    
    #duration+=end - start
    total_route+=result[-1]
    routes.append(result)
end = time.time()
duration=end-start

In [10]:
total_route, duration

(224490.34900000005, 171.95471811294556)

In [14]:
lst_clusters=lst
delta_P=0
P=0.6
N=1000
T0=0.1
K=200
alpha=0.9

#optimizing hyperparameters with grid search
lst_alpha=[0.9,0.95,0.99]
lst_P=[0.4,0.5,0.6]
lst_results={}
for alpha in lst_alpha:
    for P in lst_P:
        result_run=run_SA(lst_clusters,P,delta_P,alpha,K,T0,N)
        lst_results.update({(alpha,P):result_run})
end=time.time()

In [15]:
#get hyperparameters and corresponding routes
res=[]
for key, value in lst_results.items():
    res.append((key,value[0]))

In [16]:
res

[((0.9, 0.4), 222385.605),
 ((0.9, 0.5), 221346.42799999999),
 ((0.9, 0.6), 224490.34900000005),
 ((0.95, 0.4), 222385.605),
 ((0.95, 0.5), 221346.42799999999),
 ((0.95, 0.6), 224490.34900000005),
 ((0.99, 0.4), 222385.605),
 ((0.99, 0.5), 221346.42799999999),
 ((0.99, 0.6), 224490.34900000005)]